In [2]:
import torch 
import torch.nn as nn

from torch.autograd import Variable
import numpy as np


# Hyper Parameters

input_size = 13
hidden_size = 60
num_layers = 2
num_classes = 2
batch_size = 1
num_epochs = 2
learning_rate = 0.01

In [3]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav



In [4]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
       # print (out.size())
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

In [6]:
# create a many to one LSTM , just take last output timestep as output 
rnn = RNN(input_size, hidden_size, num_layers, num_classes)

In [7]:
# Loss and Optimizer
#criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [8]:
mfcc1 = audio_to_mfcc('/home/saurabh/Documents/audio_classification/data/lizzie.wav')
#print(mfcc1.shape)
mfcc2 = audio_to_mfcc('/home/saurabh/Documents/audio_classification/data/boy.wav')
#print(mfcc2.shape)



temp =  mfcc1[ : , np.newaxis , :]
temp2=  mfcc2[ : , np.newaxis , :]

#print(temp2.shape)
input_var = Variable(torch.Tensor(temp))
input2_var = Variable(torch.Tensor(temp2))

for epoch in range(10):
    outputs = rnn(input_var)
    outputs2= rnn(input2_var)
   # print(outputs[999])

    t1=outputs[999]
    t1=t1.unsqueeze(0)
  #  print(t1)
    
    t2=outputs2[998]
    t2=t2.unsqueeze(0)
  #  print(t2)
                    
    #final_output = Variable( t1 , requires_grad =True)
    #final_output2= Variable(  t2 , requires_grad =True)
    
   # final_output.unsqueeze(0)
   # final_output2.unsqueeze(0)
    
   # final_output_numpy=final_output.data.numpy()[np.newaxis,:]
   # final_output = torch.from_numpy(final_output_numpy)
    
   # final_output_numpy2=final_output2.data.numpy()[np.newaxis,:]
   # final_output2 = torch.from_numpy(final_output_numpy2)
    
#print(final_output_numpy.shape)

   # print(final_output.size())

    label = Variable(torch.LongTensor([0]))
    label2 = Variable(torch.LongTensor([1]))
#print (label.size())
    loss = criterion( t1, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.data[0])
    
    loss2=criterion( t2, label2)
    optimizer.zero_grad()
    loss2.backward()
    optimizer.step()
    
    print(loss2.data[0])
    
    


0.6428278684616089
0.7358896732330322
0.475163996219635
0.7055583000183105
0.2999621033668518
0.5358971953392029
0.13674023747444153
0.3297620117664337
0.047070927917957306
0.16023163497447968
0.01566590927541256
0.06110032647848129
0.005697252228856087
0.018987460061907768
0.0023355071898549795
0.005547746084630489
0.001067026169039309
0.0017001732485368848
0.0005294483271427453
0.0005983618320897222


In [17]:
# testing trained network 

mfcc3 = audio_to_mfcc('/home/saurabh/Documents/audio_classification/test/boy.wav')
temp3 =  mfcc3[ : , np.newaxis , :]

input_var3 = Variable(torch.Tensor(temp3))

outputs3 = rnn(input_var3)

print(outputs3[998])

Variable containing:
 0.1091
-0.2374
[torch.FloatTensor of size 2]

